In [1]:
## This file implements neural networks and logistic regression on p0006presabs_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 95.06% for combination data, and 82.69% for over-sampling data after improvememt.
## For logistic regression, the accuracy is 96.72% for combination data, and 75.96% for over-sampling data.
## We further construct random forest models, which are relatively less likely to bring overfitting 
## compared to decision tree.
## For random forest, the accuracy is 100% for combination data, and 85.58% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 97.86% for combination data, and 75.6% for over-sampling.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0006presabs_quant.csv')
df.shape

(255, 86)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0.537000
1      0.325833
2      0.630333
3      0.422500
4      0.433833
5      0.505833
6      0.431167
7      0.228333
8      0.547167
9      0.436667
10     0.366333
11     0.903500
12     0.431667
13     0.327500
14     0.339833
15     0.217000
16     0.441333
17     0.442667
18     0.794833
19     0.215000
20     0.426167
21     0.611667
22     0.322833
23     0.219667
24     1.004333
25     0.175000
26     0.837667
27     0.184667
28     0.195833
29     0.711667
         ...   
225    0.546400
226    0.545700
227    0.606100
228    0.964600
229    0.775600
230    0.441500
231    0.356167
232    0.437167
233    0.698667
234    0.455333
235    0.914500
236    0.231167
237    0.213667
238    0.341833
239    0.361667
240    0.342833
241    0.650667
242    0.287667
243    0.301167
244    0.514333
245    0.232500
246    0.429833
247    0.335667
248    0.191500
249    0.327667
250    0.449167
251    0.306167
252    0.388500
253    0.374833
254    0.330833
Name: pheno, Length: 255

In [6]:
df.head()

,id,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,...,group_7850,group_7871,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.537000
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.325833
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.630333
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.422500
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.433833


In [7]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [8]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [9]:
df['pheno']
df['pheno'].value_counts()

0    173
1     82
Name: pheno, dtype: int64

In [10]:
df.shape

(255, 86)

In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 85)

In [13]:
df_clean.head()

,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,GAGTCCTGTT,...,group_7850,group_7871,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [14]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 84) (255,)


In [15]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 128), (1, 75)]


In [16]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 173), (1, 173)]


In [17]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [18]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [19]:
############# Fully-Connected Neural Network ################

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [21]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [22]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 142 samples, validate on 61 samples
Epoch 1/100
142/142 [==============================] - 1s 4ms/step - loss: 0.5281 - accuracy: 0.9085 - val_loss: 0.5228 - val_accuracy: 0.8525
Epoch 2/100
142/142 [==============================] - 0s 303us/step - loss: 0.5034 - accuracy: 0.9085 - val_loss: 0.5022 - val_accuracy: 0.8689
Epoch 3/100
142/142 [==============================] - 0s 98us/step - loss: 0.4846 - accuracy: 0.9014 - val_loss: 0.4867 - val_accuracy: 0.8689
Epoch 4/100
142/142 [==============================] - 0s 110us/step - loss: 0.4694 - accuracy: 0.9085 - val_loss: 0.4722 - val_accuracy: 0.8689
Epoch 5/100
142/142 [==============================] - 0s 101us/step - loss: 0.4553 - accuracy: 0.9085 - val_loss: 0.4601 - val_accuracy: 0.8689
Epoch 6/100
142/142 [==============================] - 0s 124us/step - loss: 0.4443 - accuracy: 0.9085 - val_loss: 0.4493 - val_accuracy: 0.8689
Epoch 7/100
142/142 [==============================] - 0s 554us/step - loss: 0.4336 - ac

142/142 [==============================] - 0s 206us/step - loss: 0.2215 - accuracy: 0.9648 - val_loss: 0.2404 - val_accuracy: 0.9672
Epoch 57/100
142/142 [==============================] - 0s 158us/step - loss: 0.2195 - accuracy: 0.9648 - val_loss: 0.2387 - val_accuracy: 0.9672
Epoch 58/100
142/142 [==============================] - 0s 399us/step - loss: 0.2178 - accuracy: 0.9648 - val_loss: 0.2369 - val_accuracy: 0.9672
Epoch 59/100
142/142 [==============================] - 0s 264us/step - loss: 0.2156 - accuracy: 0.9648 - val_loss: 0.2352 - val_accuracy: 0.9672
Epoch 60/100
142/142 [==============================] - 0s 159us/step - loss: 0.2139 - accuracy: 0.9648 - val_loss: 0.2335 - val_accuracy: 0.9672
Epoch 61/100
142/142 [==============================] - 0s 380us/step - loss: 0.2122 - accuracy: 0.9648 - val_loss: 0.2320 - val_accuracy: 0.9672
Epoch 62/100
142/142 [==============================] - 0s 171us/step - loss: 0.2105 - accuracy: 0.9718 - val_loss: 0.2306 - val_accuracy

In [24]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

61/61 [==============================] - 0s 71us/step
combination test accuracy: 95.08%


In [25]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [26]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 242 samples, validate on 104 samples
Epoch 1/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7116 - accuracy: 0.5000 - val_loss: 0.6999 - val_accuracy: 0.4808
Epoch 2/100
242/242 [==============================] - 0s 236us/step - loss: 0.6888 - accuracy: 0.4835 - val_loss: 0.6861 - val_accuracy: 0.5385
Epoch 3/100
242/242 [==============================] - 0s 280us/step - loss: 0.6755 - accuracy: 0.5331 - val_loss: 0.6771 - val_accuracy: 0.6538
Epoch 4/100
242/242 [==============================] - 0s 301us/step - loss: 0.6657 - accuracy: 0.6446 - val_loss: 0.6698 - val_accuracy: 0.6058
Epoch 5/100
242/242 [==============================] - 0s 193us/step - loss: 0.6581 - accuracy: 0.6694 - val_loss: 0.6646 - val_accuracy: 0.6154
Epoch 6/100
242/242 [==============================] - 0s 131us/step - loss: 0.6519 - accuracy: 0.6488 - val_loss: 0.6600 - val_accuracy: 0.6250
Epoch 7/100
242/242 [==============================] - 0s 151us/step - loss: 0.6462 - 

Epoch 57/100
242/242 [==============================] - 0s 150us/step - loss: 0.5203 - accuracy: 0.7438 - val_loss: 0.5538 - val_accuracy: 0.6731
Epoch 58/100
242/242 [==============================] - 0s 232us/step - loss: 0.5190 - accuracy: 0.7397 - val_loss: 0.5522 - val_accuracy: 0.6731
Epoch 59/100
242/242 [==============================] - 0s 181us/step - loss: 0.5182 - accuracy: 0.7438 - val_loss: 0.5516 - val_accuracy: 0.6731
Epoch 60/100
242/242 [==============================] - 0s 117us/step - loss: 0.5164 - accuracy: 0.7479 - val_loss: 0.5507 - val_accuracy: 0.6731
Epoch 61/100
242/242 [==============================] - 0s 111us/step - loss: 0.5151 - accuracy: 0.7479 - val_loss: 0.5486 - val_accuracy: 0.6731
Epoch 62/100
242/242 [==============================] - 0s 137us/step - loss: 0.5137 - accuracy: 0.7438 - val_loss: 0.5474 - val_accuracy: 0.6731
Epoch 63/100
242/242 [==============================] - 0s 140us/step - loss: 0.5127 - accuracy: 0.7438 - val_loss: 0.5469 -

In [28]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

104/104 [==============================] - 0s 77us/step
over-sampling test accuracy: 71.15%


In [47]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [48]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [49]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 242 samples, validate on 104 samples
Epoch 1/1000
242/242 [==============================] - 0s 600us/step - loss: 0.6954 - accuracy: 0.4876 - val_loss: 0.6930 - val_accuracy: 0.5096
Epoch 2/1000
242/242 [==============================] - 0s 64us/step - loss: 0.6916 - accuracy: 0.5083 - val_loss: 0.6899 - val_accuracy: 0.5577
Epoch 3/1000
242/242 [==============================] - 0s 120us/step - loss: 0.6881 - accuracy: 0.5289 - val_loss: 0.6870 - val_accuracy: 0.5673
Epoch 4/1000
242/242 [==============================] - 0s 74us/step - loss: 0.6845 - accuracy: 0.5331 - val_loss: 0.6842 - val_accuracy: 0.5673
Epoch 5/1000
242/242 [==============================] - 0s 111us/step - loss: 0.6812 - accuracy: 0.5455 - val_loss: 0.6815 - val_accuracy: 0.5673
Epoch 6/1000
242/242 [==============================] - 0s 98us/step - loss: 0.6781 - accuracy: 0.5455 - val_loss: 0.6788 - val_accuracy: 0.5769
Epoch 7/1000
242/242 [==============================] - 0s 98us/step - loss: 0.67

242/242 [==============================] - 0s 80us/step - loss: 0.5720 - accuracy: 0.7066 - val_loss: 0.5921 - val_accuracy: 0.7308
Epoch 57/1000
242/242 [==============================] - 0s 110us/step - loss: 0.5698 - accuracy: 0.7149 - val_loss: 0.5904 - val_accuracy: 0.7308
Epoch 58/1000
242/242 [==============================] - 0s 76us/step - loss: 0.5679 - accuracy: 0.6983 - val_loss: 0.5890 - val_accuracy: 0.7308
Epoch 59/1000
242/242 [==============================] - 0s 96us/step - loss: 0.5667 - accuracy: 0.7273 - val_loss: 0.5872 - val_accuracy: 0.7308
Epoch 60/1000
242/242 [==============================] - 0s 85us/step - loss: 0.5638 - accuracy: 0.7190 - val_loss: 0.5854 - val_accuracy: 0.7308
Epoch 61/1000
242/242 [==============================] - 0s 100us/step - loss: 0.5618 - accuracy: 0.7107 - val_loss: 0.5838 - val_accuracy: 0.7308
Epoch 62/1000
242/242 [==============================] - 0s 82us/step - loss: 0.5599 - accuracy: 0.7231 - val_loss: 0.5822 - val_accurac

242/242 [==============================] - 0s 86us/step - loss: 0.4834 - accuracy: 0.7727 - val_loss: 0.5298 - val_accuracy: 0.7692
Epoch 113/1000
242/242 [==============================] - 0s 85us/step - loss: 0.4821 - accuracy: 0.7727 - val_loss: 0.5299 - val_accuracy: 0.7692
Epoch 114/1000
242/242 [==============================] - 0s 75us/step - loss: 0.4810 - accuracy: 0.7727 - val_loss: 0.5285 - val_accuracy: 0.7692
Epoch 115/1000
242/242 [==============================] - 0s 103us/step - loss: 0.4801 - accuracy: 0.7727 - val_loss: 0.5289 - val_accuracy: 0.7692
Epoch 116/1000
242/242 [==============================] - 0s 145us/step - loss: 0.4797 - accuracy: 0.7727 - val_loss: 0.5279 - val_accuracy: 0.7692
Epoch 117/1000
242/242 [==============================] - 0s 170us/step - loss: 0.4790 - accuracy: 0.7727 - val_loss: 0.5274 - val_accuracy: 0.7692
Epoch 118/1000
242/242 [==============================] - 0s 183us/step - loss: 0.4783 - accuracy: 0.7727 - val_loss: 0.5280 - val

Epoch 168/1000
242/242 [==============================] - 0s 83us/step - loss: 0.4462 - accuracy: 0.7810 - val_loss: 0.5044 - val_accuracy: 0.7788
Epoch 169/1000
242/242 [==============================] - 0s 82us/step - loss: 0.4455 - accuracy: 0.7810 - val_loss: 0.5020 - val_accuracy: 0.7788
Epoch 170/1000
242/242 [==============================] - 0s 81us/step - loss: 0.4475 - accuracy: 0.7810 - val_loss: 0.5021 - val_accuracy: 0.7788
Epoch 171/1000
242/242 [==============================] - 0s 80us/step - loss: 0.4452 - accuracy: 0.7810 - val_loss: 0.5015 - val_accuracy: 0.7788
Epoch 172/1000
242/242 [==============================] - 0s 74us/step - loss: 0.4459 - accuracy: 0.7810 - val_loss: 0.4998 - val_accuracy: 0.7692
Epoch 173/1000
242/242 [==============================] - 0s 77us/step - loss: 0.4441 - accuracy: 0.7810 - val_loss: 0.5008 - val_accuracy: 0.7788
Epoch 174/1000
242/242 [==============================] - 0s 80us/step - loss: 0.4429 - accuracy: 0.7810 - val_loss: 0

Epoch 224/1000
242/242 [==============================] - 0s 80us/step - loss: 0.4209 - accuracy: 0.7893 - val_loss: 0.4756 - val_accuracy: 0.7692
Epoch 225/1000
242/242 [==============================] - 0s 78us/step - loss: 0.4198 - accuracy: 0.7934 - val_loss: 0.4745 - val_accuracy: 0.7692
Epoch 226/1000
242/242 [==============================] - 0s 78us/step - loss: 0.4195 - accuracy: 0.7934 - val_loss: 0.4746 - val_accuracy: 0.7692
Epoch 227/1000
242/242 [==============================] - 0s 80us/step - loss: 0.4189 - accuracy: 0.8017 - val_loss: 0.4797 - val_accuracy: 0.7788
Epoch 228/1000
242/242 [==============================] - 0s 81us/step - loss: 0.4193 - accuracy: 0.7934 - val_loss: 0.4796 - val_accuracy: 0.7788
Epoch 229/1000
242/242 [==============================] - 0s 78us/step - loss: 0.4209 - accuracy: 0.7851 - val_loss: 0.4758 - val_accuracy: 0.7788
Epoch 230/1000
242/242 [==============================] - 0s 79us/step - loss: 0.4193 - accuracy: 0.7851 - val_loss: 0

...

Epoch 727/1000
242/242 [==============================] - 0s 81us/step - loss: 0.2765 - accuracy: 0.8636 - val_loss: 0.3837 - val_accuracy: 0.8173
Epoch 728/1000
242/242 [==============================] - 0s 108us/step - loss: 0.2758 - accuracy: 0.8719 - val_loss: 0.3814 - val_accuracy: 0.8750
Epoch 729/1000
242/242 [==============================] - 0s 69us/step - loss: 0.2898 - accuracy: 0.8719 - val_loss: 0.3780 - val_accuracy: 0.8365
Epoch 730/1000
242/242 [==============================] - 0s 89us/step - loss: 0.2811 - accuracy: 0.8678 - val_loss: 0.3794 - val_accuracy: 0.8750
Epoch 731/1000
242/242 [==============================] - 0s 88us/step - loss: 0.2817 - accuracy: 0.8636 - val_loss: 0.3868 - val_accuracy: 0.8173
Epoch 732/1000
242/242 [==============================] - 0s 92us/step - loss: 0.2766 - accuracy: 0.8636 - val_loss: 0.3761 - val_accuracy: 0.8269
Epoch 733/1000
242/242 [==============================] - 0s 86us/step - loss: 0.2783 - accuracy: 0.8512 - val_loss: 

Epoch 783/1000
242/242 [==============================] - 0s 84us/step - loss: 0.2680 - accuracy: 0.8719 - val_loss: 0.3871 - val_accuracy: 0.8750
Epoch 784/1000
242/242 [==============================] - 0s 80us/step - loss: 0.2746 - accuracy: 0.8554 - val_loss: 0.3863 - val_accuracy: 0.8750
Epoch 785/1000
242/242 [==============================] - 0s 91us/step - loss: 0.2802 - accuracy: 0.8512 - val_loss: 0.3808 - val_accuracy: 0.8269
Epoch 786/1000
242/242 [==============================] - 0s 77us/step - loss: 0.2840 - accuracy: 0.8678 - val_loss: 0.3830 - val_accuracy: 0.8750
Epoch 787/1000
242/242 [==============================] - 0s 91us/step - loss: 0.2799 - accuracy: 0.8636 - val_loss: 0.3835 - val_accuracy: 0.8077
Epoch 788/1000
242/242 [==============================] - 0s 78us/step - loss: 0.2737 - accuracy: 0.8719 - val_loss: 0.3990 - val_accuracy: 0.8173
Epoch 789/1000
242/242 [==============================] - 0s 79us/step - loss: 0.2798 - accuracy: 0.8636 - val_loss: 0

Epoch 839/1000
242/242 [==============================] - 0s 84us/step - loss: 0.2725 - accuracy: 0.8430 - val_loss: 0.3889 - val_accuracy: 0.8077
Epoch 840/1000
242/242 [==============================] - 0s 80us/step - loss: 0.2691 - accuracy: 0.8719 - val_loss: 0.3849 - val_accuracy: 0.8077
Epoch 841/1000
242/242 [==============================] - 0s 79us/step - loss: 0.2713 - accuracy: 0.8760 - val_loss: 0.3866 - val_accuracy: 0.8077
Epoch 842/1000
242/242 [==============================] - 0s 78us/step - loss: 0.2685 - accuracy: 0.8719 - val_loss: 0.3907 - val_accuracy: 0.8750
Epoch 843/1000
242/242 [==============================] - 0s 78us/step - loss: 0.2699 - accuracy: 0.8554 - val_loss: 0.3895 - val_accuracy: 0.8750
Epoch 844/1000
242/242 [==============================] - 0s 82us/step - loss: 0.2785 - accuracy: 0.8678 - val_loss: 0.3984 - val_accuracy: 0.8750
Epoch 845/1000
242/242 [==============================] - 0s 85us/step - loss: 0.2758 - accuracy: 0.8802 - val_loss: 0

Epoch 895/1000
242/242 [==============================] - 0s 80us/step - loss: 0.2728 - accuracy: 0.8678 - val_loss: 0.3932 - val_accuracy: 0.8077
Epoch 896/1000
242/242 [==============================] - 0s 85us/step - loss: 0.2678 - accuracy: 0.8760 - val_loss: 0.3977 - val_accuracy: 0.8077
Epoch 897/1000
242/242 [==============================] - 0s 78us/step - loss: 0.2637 - accuracy: 0.8678 - val_loss: 0.3961 - val_accuracy: 0.8750
Epoch 898/1000
242/242 [==============================] - 0s 76us/step - loss: 0.2681 - accuracy: 0.8636 - val_loss: 0.3924 - val_accuracy: 0.8269
Epoch 899/1000
242/242 [==============================] - 0s 78us/step - loss: 0.2657 - accuracy: 0.8554 - val_loss: 0.3951 - val_accuracy: 0.8750
Epoch 900/1000
242/242 [==============================] - 0s 77us/step - loss: 0.2710 - accuracy: 0.8636 - val_loss: 0.3926 - val_accuracy: 0.8077
Epoch 901/1000
242/242 [==============================] - 0s 76us/step - loss: 0.2621 - accuracy: 0.8760 - val_loss: 0

Epoch 951/1000
242/242 [==============================] - 0s 78us/step - loss: 0.2651 - accuracy: 0.8554 - val_loss: 0.4089 - val_accuracy: 0.8173
Epoch 952/1000
242/242 [==============================] - 0s 77us/step - loss: 0.2633 - accuracy: 0.8678 - val_loss: 0.3957 - val_accuracy: 0.8077
Epoch 953/1000
242/242 [==============================] - 0s 78us/step - loss: 0.2664 - accuracy: 0.8595 - val_loss: 0.4132 - val_accuracy: 0.8173
Epoch 954/1000
242/242 [==============================] - 0s 76us/step - loss: 0.2671 - accuracy: 0.8595 - val_loss: 0.4011 - val_accuracy: 0.8077
Epoch 955/1000
242/242 [==============================] - 0s 81us/step - loss: 0.2682 - accuracy: 0.8760 - val_loss: 0.3955 - val_accuracy: 0.7981
Epoch 956/1000
242/242 [==============================] - 0s 89us/step - loss: 0.2588 - accuracy: 0.8760 - val_loss: 0.3940 - val_accuracy: 0.7981
Epoch 957/1000
242/242 [==============================] - 0s 81us/step - loss: 0.2614 - accuracy: 0.8595 - val_loss: 0

In [50]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

104/104 [==============================] - 0s 153us/step
over-sampling test accuracy: 82.69%


In [33]:
############ Logistic Regression ############

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [35]:
###### logistics on combination data
log_comb = LogisticRegression(random_state=123)
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [36]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 96.72%


In [37]:
##### logistics on over-sampling data
log_over = LogisticRegression(random_state=123)
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [38]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 75.96%


In [39]:
############## Random Forest ##############

In [40]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100,random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [41]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [42]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [43]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 85.58%


In [44]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [45]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         1.         0.92857143 1.         0.96428571]
0.9785714285714286


In [46]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.78       0.83333333 0.70833333 0.6875     0.77083333]
0.756
